#Ładowanie zależności

In [1]:
from PIL import Image
import cv2
import csv
import numpy as np
import math
import os
from numpy import loadtxt
import random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import pandas as pd


Generowanie klas, config

In [2]:
class Config():
    def __init__(self):
        self.noise_generation=True #fflag of noise photos generation
        self.noise_count=10 #number of generated noise images
        self.generateNoisedImages=True #false=unnoised image generator, true=noisedImageGenerator
        self.imagesToGenerateNumber=10

        self.measured_dataset_metadata = "../data/raw/1channel/reference/epsilon.csv" 
        self.measured_data_root_dir = "../data/raw/1channel/photo"
        ####################################
        #NoiseGeneration
        self.noise_root_dir="../data/generated/noise/photo"
        self.noise_root_metadata=""                                                 #TODO
        
        self.noise_minimum_foto_count=150
        self.noise_maximum_foto_count=250   #min and max number of measured photos to calculate noise
        ####################################
        #Image generation
        self.w = 640 #width
        self.h = 480 #height
        self.ringsDifference=self.w*self.w/6 #Difference in squared distance from center between succesive stripes
        #57 600 #68 266
        self.middleXMaxVariation=50 #in pixels
        self.middleYMaxVariation=50 #in pisels
        self.epsilonMin=0
        self.epsilonMax=0.9999
        self.pixelBrightnessMaxNoise=5 #in number, +- that valueself.
        self.pixelMaxUpperLimit=210  #upper limit max brightness of pixel, 8bit-> 0-255
        self.pixelMaxLowerLimit=170  #lower limit max brightness of pixel, 8bit-> 0-255self.
        self.pixelMinUpperLimit=120 #upper limit min brightness of pixe, 8bit-> 0-255
        self.pixelMinLowerLimit=80 #lower limit min brightness of pixe, 8bit-> 0-255self.
        self.middleX=self.w/2 #Middle of centre, X axis
        self.middleY=self.h/2 #Middle of centre, Y axis
        self.generated_noised_photo_data_root_dir=str(Path().resolve().parent)+"\\data\\generated\\noised\photo"
        self.generated_noised_photo_metadata=str(Path().resolve().parent)+"\\data\\generated\\noised\\reference\\epsilon.csv"
        self.generated_unnoised_photo_data_root_dir=str(Path().resolve().parent)+"\\data\\generated\\unnoised\photo"
        self.generated_unnoised_photo_metadata=str(Path().resolve().parent)+"\\data\\generated\\unnoised\\reference\\epsilon.csv"
        self.noiseFilesPaths=[]


class EpsilonDataset(torch.utils.data.Dataset):
   def __init__(self, root_dir, annotation_file, transform=None):
       self.root_dir = root_dir
       self.annotations = pd.read_csv(annotation_file,skiprows=0, delim_whitespace=' ')
       self.transform = transform
   def __len__(self):
       return len(self.annotations)
   def __getitem__(self, index):
       img = np.array(cv2.imread(os.path.join(self.root_dir, str(str("%05d" %self.annotations.imgnr[index]))+ ".png"))).astype(np.float32)
       #img=PIL.Image.fromarray(np.uint8(img)) #In transferLearning it has to be PIL image, but in noise generation, it is better to use numpy arrays
       y_label = self.annotations.eps[index]
       if self.transform is not None:
           img = self.transform(img)
       return img, y_label   

    


In [3]:
cfg=Config()
measuredDataset= EpsilonDataset(cfg.measured_data_root_dir, cfg.measured_dataset_metadata)


def noiseOnePhotoGenerator(fotoCountMin, fotoCountMax, noiseGenerationCount, imageDataset, noiseRootDir):
    fotoCount=random.randint(fotoCountMin, fotoCountMax)
    os.makedirs(noiseRootDir, exist_ok=True)
    Combined=np.zeros((480,640))
    for noiseNumber in range (noiseGenerationCount):        
        for i in range(fotoCount):
            randomNumber=random.randint(0,len(imageDataset)-1)
            img, label=imageDataset[randomNumber]   
            img=img[4:484,4:644,0]
            imgMean=np.mean(img)
            Combined=(Combined+img)

        Combined=Combined/fotoCount
        Combined=Combined.astype(np.uint8)

        Noise=Image.fromarray(Combined)
        Noise.save(noiseRootDir+"/"+str(noiseNumber)+".png")
        if (noiseNumber%10==0): print("Made ",noiseNumber,"noise")

if(cfg.noise_generation==True):
    noiseOnePhotoGenerator(fotoCountMin= cfg.noise_minimum_foto_count, fotoCountMax= cfg.noise_maximum_foto_count, 
                                noiseGenerationCount= cfg.noise_count, imageDataset= measuredDataset, noiseRootDir= cfg.noise_root_dir)


Made  0 noise


In [4]:
cfg=Config()
def GenerateImage(w,h,middleXPos,middleYPos,ringsDifference, epsilonMin, epsilonMax,
                  pixelBrightnessMaxNoise, noiseFilesPaths, middleXMaxVariation, middleYMaxVariation, 
                  pixelMaxLowerLimit, pixelMaxUpperLimit, pixelMinLowerLimit, pixelMinUpperLimit,
                  generateNoisedImages):
  epsilon=random.randint(epsilonMin*10000,epsilonMax*10000)/10000.0
  middleXVariation=random.randint(-middleXMaxVariation, middleXMaxVariation)
  middleYVariation=random.randint(-middleYMaxVariation, middleYMaxVariation)
  pixelMax=random.randint(pixelMaxLowerLimit, pixelMaxUpperLimit)
  pixelMin=random.randint(pixelMinLowerLimit, pixelMinUpperLimit)
  middleX=middleXPos+middleXVariation
  middleY=middleYPos+middleYVariation
  pixelMean=(pixelMax+pixelMin)/2
  pixelDiff=pixelMax-pixelMean
  pixelNoise=random.randint(-pixelBrightnessMaxNoise, pixelBrightnessMaxNoise)
  data=np.ones((h,w))
  data=data[:,:]*pixelMean
  for x in range (w):
    for y in range(h):
      data[y,x]=data[y,x]+(pixelDiff*math.cos( 2*math.pi*( epsilon+ ((pow((x-middleX)*2,2)+pow((y-middleY)*2,2)) /ringsDifference))))+pixelNoise
    #print("x: ",x,", y:  ",y,", data:  ",data[x,y], "liczba " , 2*math.pi*( ((pow((x-w/2)*2,2)+pow((y-h/2)*2,2)) /RingsDifference)))
  data=data.astype(np.uint8)
  if(generateNoisedImages==True): return Noising(data, noiseFilesPaths, epsilon)
  else:
      data=data.astype(np.uint8)
      data=Image.fromarray(data) 
      return data, epsilon

def Noising(generated, noiseFilesPaths, epsilon):
    randomNoiseInterator=random.randint(0,len(noiseFilesPaths)-1)
    noise=np.array(cv2.imread(noiseFilesPaths[randomNoiseInterator]))
    noise=noise[:,:,0]
    noiseMean=np.mean(noise)
    differ=-(noise-noiseMean)
    generated=generated[:,:]
    generated=generated-differ
    generated=np.clip(generated,0,255)
    generated=generated.astype(np.uint8)
    noised=Image.fromarray(generated)
    return noised, epsilon
    #noised.save("NoisedImage.png")

def writeToFile(filePath, row):
    f=open(filePath, "a", encoding='UTF8')
    f.write(row)
    f.close()

def makedirectories(cfg):
    os.makedirs(cfg.generated_noised_photo_data_root_dir, exist_ok=True)
    os.makedirs(cfg.generated_unnoised_photo_data_root_dir, exist_ok=True)
      
    if(os.path.isfile(cfg.generated_noised_photo_metadata)==False):
        os.makedirs(cfg.generated_noised_photo_metadata, exist_ok=True)
        os.rmdir(cfg.generated_noised_photo_metadata)
        writeToFile(cfg.generated_noised_photo_metadata,"imgnr  eps \n")
    if(os.path.isfile(cfg.generated_unnoised_photo_metadata)==False):
        os.makedirs(cfg.generated_unnoised_photo_metadata, exist_ok=True)
        os.rmdir(cfg.generated_unnoised_photo_metadata)
        writeToFile(cfg.generated_unnoised_photo_metadata,"imgnr  eps \n")

def createNoiseFilesPathList():
    noiseFilesPaths=[]
    for filename in os.listdir(cfg.noise_root_dir):
        #img = cv2.imread(os.path.join(dir,filename))
        #if img is not None:
        if(filename.__contains__(".png") ): noiseFilesPaths.append(str(cfg.noise_root_dir)+"/"+str(filename))
    return noiseFilesPaths

def selectPathsForGeneratedFiles(generateNoisedImages, noisedRootDir, noisedMetadataDir, unnoisedRootDir, unnoisedMetadadaDir):
    csvFilePath=""
    photoSavePath=""
    if(generateNoisedImages==True):
        csvFilePath=noisedMetadataDir
        photoSavePath=noisedRootDir
    else:
        csvFilePath=unnoisedMetadadaDir
        photoSavePath=unnoisedRootDir
    photosNumberInSavePath=len(os.listdir(photoSavePath))
    return csvFilePath, photoSavePath, photosNumberInSavePath



makedirectories(cfg)
cfg.noiseFilesPaths=createNoiseFilesPathList()

csvFilePath, photoSavePath, photosNumberInSavePath=selectPathsForGeneratedFiles(
            generateNoisedImages=cfg.generateNoisedImages, 
            noisedMetadataDir=cfg.generated_noised_photo_metadata, noisedRootDir=cfg.generated_noised_photo_data_root_dir,
            unnoisedMetadadaDir= cfg.generated_unnoised_photo_metadata, unnoisedRootDir=cfg.generated_unnoised_photo_data_root_dir)

for i in range(cfg.imagesToGenerateNumber):
    img, epsilon=GenerateImage(w= cfg.w, h=cfg.h, middleXPos=cfg.middleX, middleYPos=cfg.middleY,
                      ringsDifference=cfg.ringsDifference, epsilonMin=cfg.epsilonMin, epsilonMax=cfg.epsilonMax, 
                      pixelBrightnessMaxNoise= cfg.pixelBrightnessMaxNoise, noiseFilesPaths=cfg.noiseFilesPaths,
                      middleXMaxVariation=cfg.middleXMaxVariation, middleYMaxVariation= cfg.middleYMaxVariation,
                      pixelMaxLowerLimit=cfg.pixelMaxLowerLimit, pixelMaxUpperLimit=cfg.pixelMaxUpperLimit,
                      pixelMinLowerLimit=cfg.pixelMinLowerLimit, pixelMinUpperLimit=cfg.pixelMinUpperLimit,
                      generateNoisedImages= cfg.generateNoisedImages)
    
    img.save(os.path.join(photoSavePath,str(str("%05d" %(i+photosNumberInSavePath)) +".png")))
    writeToFile(csvFilePath,str(i+photosNumberInSavePath)+" "+ str(epsilon)+"\n" )
    if(i%10==0): print("Generated",i, "image")



Generated 0 image
Generated 10 image
Generated 20 image
Generated 30 image
Generated 40 image
Generated 50 image
Generated 60 image
Generated 70 image
Generated 80 image
Generated 90 image
